# ML on NYCFlights with PySpark ML

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, dayofweek, dayofmonth, month
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/07 21:44:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load the data

In [39]:
flights = spark.read.parquet("../data/flights.parquet")
weather = spark.read.parquet("../data/weather.parquet")

In [40]:
flights.show(10)

+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|
+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+
|2013|    1|  1|     517|           515|      2.0|     830|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|  1400.0| 5.0|  15.0|2013-01-01 10:00:00|
|2013|    1|  1|     533|           529|      4.0|     850|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|  1416.0| 5.0|  29.0|2013-01-01 10:00:00|
|2013|    1|  1|     542|           540|      2.0|     923|           850|     33.0|     AA|  1141| N619AA|   JFK| MIA|   160.0|  1089.0| 5.0|  40.0|2

In [41]:
weather.show(10)

+------+----+-----+---+----+-----+-----+-----+--------+------------------+---------+------+--------+-----+-------------------+
|origin|year|month|day|hour| temp| dewp|humid|wind_dir|        wind_speed|wind_gust|precip|pressure|visib|          time_hour|
+------+----+-----+---+----+-----+-----+-----+--------+------------------+---------+------+--------+-----+-------------------+
|   EWR|2013|    1|  1|   1|39.02|26.06|59.37|   270.0|10.357019999999999|     null|   0.0|  1012.0| 10.0|2013-01-01 06:00:00|
|   EWR|2013|    1|  1|   2|39.02|26.96|61.63|   250.0|           8.05546|     null|   0.0|  1012.3| 10.0|2013-01-01 07:00:00|
|   EWR|2013|    1|  1|   3|39.02|28.04|64.43|   240.0|           11.5078|     null|   0.0|  1012.5| 10.0|2013-01-01 08:00:00|
|   EWR|2013|    1|  1|   4|39.92|28.04|62.21|   250.0|12.658579999999999|     null|   0.0|  1012.2| 10.0|2013-01-01 09:00:00|
|   EWR|2013|    1|  1|   5|39.02|28.04|64.43|   260.0|12.658579999999999|     null|   0.0|  1011.9| 10.0|2013-

---

## Merge & Clean training data

In [42]:
t = (
    flights
    .withColumn("delayed", col("arr_delay") >= 30)
    .withColumn("date", col("time_hour").cast("date"))
    .join(weather, ["origin", "time_hour"])
    .select(
        "dep_time", "flight", "origin", "dest", "air_time", "distance", "carrier", "date",
        "temp", "dewp", "humid", "wind_dir", "wind_speed", "precip", "pressure", "visib",
        "delayed"
    )
    .dropna()
    .withColumn("date_dow", dayofweek(col("date")))
    .withColumn("date_month", month(col("date")))
    .withColumn("xmas_eve", when((month(col("date")) == 12) & (dayofmonth(col("date")) == 24), True).otherwise(False))
    .withColumn("xmas", when((month(col("date")) == 12) & (dayofmonth(col("date")) == 25), True).otherwise(False))
    .withColumn("july4", when((month(col("date")) == 7) & (dayofmonth(col("date")) == 4), True).otherwise(False))
    .drop("date")
)

t.show(10)

+--------+------+------+----+--------+--------+-------+-----+-----+-----+--------+------------------+------+--------+-----+-------+--------+----------+--------+-----+-----+
|dep_time|flight|origin|dest|air_time|distance|carrier| temp| dewp|humid|wind_dir|        wind_speed|precip|pressure|visib|delayed|date_dow|date_month|xmas_eve| xmas|july4|
+--------+------+------+----+--------+--------+-------+-----+-----+-----+--------+------------------+------+--------+-----+-------+--------+----------+--------+-----+-----+
|     517|  1545|   EWR| IAH|   227.0|  1400.0|     UA|39.02|28.04|64.43|   260.0|12.658579999999999|   0.0|  1011.9| 10.0|  false|       3|         1|   false|false|false|
|     533|  1714|   LGA| IAH|   227.0|  1416.0|     UA|39.92|24.98|54.81|   250.0|14.960139999999999|   0.0|  1011.4| 10.0|  false|       3|         1|   false|false|false|
|     542|  1141|   JFK| MIA|   160.0|  1089.0|     AA|39.02|26.96|61.63|   260.0|14.960139999999999|   0.0|  1012.1| 10.0|   true|    

In [44]:
(train, test) = t.randomSplit([0.2, 0.8])

---

## Build a Pipeline

In [45]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [46]:
pipeline = Pipeline(
    stages=[
        StringIndexer(
            inputCols=["origin", "dest", "carrier"],
            outputCols=["origin_indexed", "dest_indexed", "carrier_indexed"]
        )
    ]
)

In [47]:
pipeline_model = pipeline.fit(train)

In [48]:
pipeline_model.stages

[StringIndexerModel: uid=StringIndexer_caa8f3baa1f7, handleInvalid=error, numInputCols=3, numOutputCols=3]

In [49]:
result = pipeline_model.transform(train)

---

## Fit a Model

In [50]:
import xgboost as xgb

In [51]:
model = xgb.XGBClassifier(tree_method="hist", enable_categorical=True, objective="binary:hinge")

In [52]:
df = result.toPandas()
X = df.drop(["delayed", "origin", "dest", "carrier"], axis=1)
y = df["delayed"]

In [53]:
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='binary:hinge', predictor=None, ...)

---

## Evaluate the Model

In [54]:
test_result = pipeline_model.transform(test)

In [55]:
df_test = result.toPandas()
X_test = df_test.drop(["delayed", "origin", "dest", "carrier"], axis=1)
y_test = df_test["delayed"]

In [56]:
y_predict = model.predict(X_test)

In [57]:
from sklearn.metrics import accuracy_score

In [58]:
accuracy_score(y_test, y_predict)

0.9011319172722191

## Is that good?

In [59]:
import numpy as np

In [60]:
accuracy_score(y_test, np.zeros_like(y_test))

0.8550167691447736